In [ ]:
# !unrar x "/content/drive/MyDrive/Dicoding/classification.rar" "/content/drive/MyDrive/Dicoding/process"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, LeakyReLU, Flatten, TimeDistributed
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, optimizers

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score

from matplotlib import pyplot as plt
from IPython.display import clear_output, display

import pandas as pd
import numpy as np
import tensorflow as tf

import math
import glob
import os
import cv2
import sys
import time
import shutil

In [ ]:
class Config:
  def __init__(self):
    self.input_dim = 224
    self.input_shape = ( self.input_dim , self.input_dim , 3 )
    self.dropout_rate = 0.5
    self.alpha = 0.2

class DicodingCLS(Config):
  def __init__(self):
    super().__init__()

    

    self.base_dir = "/content/drive/MyDrive/Dicoding/"
    self.dataset_dir = self.base_dir + "dataset/"
    self.label = [x for x in sorted(os.listdir(self.dataset_dir))]
    print("Label : ", self.label)
    self.processing_dir = self.base_dir + "process"
    try:
      # shutil.rmtree(self.processing_dir)
      pass
    except:
      pass

    self.train_dir = os.path.join(self.processing_dir, 'train/')
    self.test_dir = os.path.join(self.processing_dir, 'test/')

    self.train_A_dir = os.path.join(self.train_dir, "A")
    self.train_B_dir = os.path.join(self.train_dir, "B")
    self.train_C_dir = os.path.join(self.train_dir, "C")
    self.train_D_dir = os.path.join(self.train_dir, "D")

    self.test_A_dir = os.path.join(self.test_dir, "A")
    self.test_B_dir = os.path.join(self.test_dir, "B")
    self.test_C_dir = os.path.join(self.test_dir, "C")
    self.test_D_dir = os.path.join(self.test_dir, "D")

    if not os.path.exists(self.processing_dir):
      print("Build New Directory")
      os.mkdir(self.processing_dir)

      
      os.mkdir(self.train_dir)
      
      os.mkdir(self.test_dir)

      
      os.mkdir(self.train_A_dir)
      
      os.mkdir(self.train_B_dir)
      
      os.mkdir(self.train_C_dir)
      
      os.mkdir(self.train_D_dir)

      
      os.mkdir(self.test_A_dir)
      
      os.mkdir(self.test_B_dir)
      
      os.mkdir(self.test_C_dir)
      
      os.mkdir(self.test_D_dir)
    else:
      print("Directory Setted")
  
  def load_image(self):
    self.num_data = {}
    self.x_train = []
    self.y_train = []
    self.x_test = []
    self.y_test  = []
    self.num_train_test = {}

    for self.ls in sorted(os.listdir(self.dataset_dir)):
      self.num_data[self.ls] = len(os.listdir(self.dataset_dir + self.ls))
    print("Total Data : ",self.num_data)
    print()

    for self.idx in range(len(self.label)):
      self.list_dir = glob.glob(os.path.join(self.dataset_dir+self.label[self.idx]+"/", '*.jpg'))
      self.num_train_test[self.label[self.idx]] = [self.list_dir[:int(0.8*len(self.list_dir))], self.list_dir[int(0.8*len(self.list_dir)):]]
    
    print("Number of Data Train-Test:")
    print()
    for x in self.num_train_test:
      self.ntrain =  np.asarray(self.num_train_test[x][0]).shape
      self.ntest = np.asarray(self.num_train_test[x][1]).shape
      print(x, self.ntrain, self.ntest)
  
  def image_to_dir(self):
    print("Copy to Train Dir")
    counter = 0
    for fname in self.num_train_test["A"][0]:
      counter += 1

      if counter == 2000:
        for i in range(1,11):
          sys.stdout.write("\r" + fname + str(" {}/{}".format(counter, len(self.num_train_test["A"][0]))) + " Wait...{} S".format(i))
          time.sleep(1)

      sys.stdout.write("\r" + fname + str(" {}/{}".format(counter, len(self.num_train_test["A"][0]))))
      src = fname
      dst = self.train_A_dir
      shutil.copy(src, dst)
    print()
    counter = 0
    for fname in self.num_train_test["B"][0]:
      counter += 1

      if counter == 2000:
        for i in range(1,11):
          sys.stdout.write("\r" + fname + str(" {}/{}".format(counter, len(self.num_train_test["A"][0]))) + " Wait...{} S".format(i))
          time.sleep(1)
        
      sys.stdout.write("\r" + fname + str(" {}/{}".format(counter, len(self.num_train_test["B"][0]))))
      src = fname
      dst = self.train_B_dir
      shutil.copy(src, dst)
    print()
    counter = 0
    for fname in self.num_train_test["C"][0]:
      counter += 1
      
      if counter == 2000:
        for i in range(1,11):
          sys.stdout.write("\r" + fname + str(" {}/{}".format(counter, len(self.num_train_test["A"][0]))) + " Wait...{} S".format(i))
          time.sleep(1)
        
      sys.stdout.write("\r" + fname + str(" {}/{}".format(counter, len(self.num_train_test["C"][0]))))
      src = fname
      dst = self.train_C_dir
      shutil.copy(src, dst)
    print()
    counter = 0
    for fname in self.num_train_test["D"][0]:
      counter += 1
      
      if counter == 2000:
        for i in range(1,11):
          sys.stdout.write("\r" + fname + str(" {}/{}".format(counter, len(self.num_train_test["A"][0]))) + " Wait...{} S".format(i))
          time.sleep(1)
        
      sys.stdout.write("\r" + fname + str(" {}/{}".format(counter, len(self.num_train_test["D"][0]))))
      src = fname
      dst = self.train_D_dir
      shutil.copy(src, dst)
    print()
    print("Copy to Test Dir")
    counter = 0
    for fname in self.num_train_test["A"][1]:
      counter += 1
      
      if counter == 2000:
        for i in range(1,11):
          sys.stdout.write("\r" + fname + str(" {}/{}".format(counter, len(self.num_train_test["A"][0]))) + " Wait...{} S".format(i))
          time.sleep(1)
        
      sys.stdout.write("\r" + fname + str(" {}/{}".format(counter, len(self.num_train_test["A"][1]))))
      src = fname
      dst = self.test_A_dir
      shutil.copy(src, dst)
    print()
    counter = 0
    for fname in self.num_train_test["B"][1]:
      counter += 1
      
      if counter == 2000:
        for i in range(1,11):
          sys.stdout.write("\r" + fname + str(" {}/{}".format(counter, len(self.num_train_test["A"][0]))) + " Wait...{} S".format(i))
          time.sleep(1)
        
      sys.stdout.write("\r" + fname + str(" {}/{}".format(counter, len(self.num_train_test["B"][1]))))
      src = fname
      dst = self.test_B_dir
      shutil.copy(src, dst)
    print()
    counter = 0
    for fname in self.num_train_test["C"][1]:
      counter += 1
      
      if counter == 2000:
        for i in range(1,11):
          sys.stdout.write("\r" + fname + str(" {}/{}".format(counter, len(self.num_train_test["A"][0]))) + " Wait...{} S".format(i))
          time.sleep(1)
        
      sys.stdout.write("\r" + fname + str(" {}/{}".format(counter, len(self.num_train_test["C"][1]))))
      src = fname
      dst = self.test_C_dir
      shutil.copy(src, dst)
    print()
    counter = 0
    for fname in self.num_train_test["D"][1]:
      counter += 1
      
      if counter == 2000:
        for i in range(1,11):
          sys.stdout.write("\r" + fname + str(" {}/{}".format(counter, len(self.num_train_test["A"][0]))) + " Wait...{} S".format(i))
          time.sleep(1)
        
      sys.stdout.write("\r" + fname + str(" {}/{}".format(counter, len(self.num_train_test["D"][1]))))
      src = fname
      dst = self.test_D_dir
      shutil.copy(src, dst)
    print()

  def start(self):
    pass
    
    # self.load_image()
    # self.image_to_dir()

CLS = DicodingCLS()
CLS.start()

Label :  ['A', 'B', 'C', 'D']
Directory Setted


In [ ]:
train = ImageDataGenerator(rescale=1./255)
test = ImageDataGenerator(rescale=1./255)

train_gen = train.flow_from_directory(
    CLS.train_dir,
    target_size=(150, 150),
    batch_size=80,
    class_mode='categorical')
test_gen = test.flow_from_directory(
    CLS.test_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='categorical')

Found 9600 images belonging to 4 classes.
Found 2400 images belonging to 4 classes.


In [ ]:
def build_neural_network():
  alpha = 0.2

  model_layers = [
      # VGG16(weights='imagenet',
      #       include_top=False,
      #       input_shape=(150, 150, 3)),
      Conv2D(64, (3, 3), activation='relu', input_shape=(150, 150, 3)),
      Conv2D(32, kernel_size=(3,3)),
      LeakyReLU(alpha = alpha),
      MaxPooling2D(pool_size=(2,2)),
      Dropout(0.5),

      Flatten(),
      Dense(16, activation = 'relu'),
      Dropout(0.5),
      Dense(4, activation = 'softmax')
  ]
  
  model = models.Sequential(model_layers)
  model.summary()
  model.compile(
      optimizer = optimizers.RMSprop(learning_rate = 5e-5),
      loss = 'categorical_crossentropy',
      metrics = ['acc']
  )
  return model

neural_model = build_neural_network()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 146, 146, 32)      18464     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 146, 146, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 73, 73, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 170528)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)               

In [ ]:
model_ckpt = ModelCheckpoint(CLS.base_dir+"CLSModel.h5", monitor = "val_acc",verbose=1, save_best_only=True, mode = 'max')
history = neural_model.fit(train_gen,
                           steps_per_epoch=9600//80,
                           epochs=10,
                           validation_data=test_gen,
                           validation_steps=2400//20,
                           callbacks = [model_ckpt])

Epoch 1/10
  2/120 [..............................] - ETA: 44:46 - loss: 1.4696 - acc: 0.2812

KeyboardInterrupt: ignored

In [ ]:
_history = pd.DataFrame({"Acc" : history.history['accuracy'],
              "Loss" : history.history['loss'],
              "Val Acc" : history.history['val_accuracy'],
              "Val Loss" : history.history['val_loss'],
              })
if os.path.exists(CLS.base_dir+"history.csv"):
  print("Append Existing History")
  history.to_csv(CLS.base_dir+'history.csv', mode='a', index=False, header=False)
else:
  print("Write New History")
  history.to_csv(CLS.base_dir+'history.csv', mode='w', index=False, header=True)

In [ ]:

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()